In [1]:
from mip import *

m = Model("Sankey")

In [2]:
# Declare the constants
wn = [
    13,
    8,
    6,
    9,
    10,
    12,
    6,
    6,
    8
]

w = {
    '03':5,
    '04':2,
    '05':6,
    '14':8,
    '25':6,
    '36':6,
    '37':3,
    '48':4,
    '57':3,
    '58':4
}

dlb = 1

In [3]:
# Create variables

y = [
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS),
    m.add_var(var_type = CONTINUOUS)
]

d = {
    'o0':m.add_var(var_type = CONTINUOUS, lb = 0),
    '01':m.add_var(var_type = CONTINUOUS, lb = dlb),
    '12':m.add_var(var_type = CONTINUOUS, lb = dlb),
    'o1':m.add_var(var_type = CONTINUOUS, lb = 0),
    '34':m.add_var(var_type = CONTINUOUS, lb = dlb),
    '45':m.add_var(var_type = CONTINUOUS, lb = dlb),
    'o2':m.add_var(var_type = CONTINUOUS, lb = 0),
    '67':m.add_var(var_type = CONTINUOUS, lb = dlb),
    '78':m.add_var(var_type = CONTINUOUS, lb = dlb)
}

s = {
    '03':m.add_var(var_type = CONTINUOUS),
    '04':m.add_var(var_type = CONTINUOUS),
    '05':m.add_var(var_type = CONTINUOUS),
    '14':m.add_var(var_type = CONTINUOUS),
    '25':m.add_var(var_type = CONTINUOUS),
    '36':m.add_var(var_type = CONTINUOUS),
    '37':m.add_var(var_type = CONTINUOUS),
    '48':m.add_var(var_type = CONTINUOUS),
    '57':m.add_var(var_type = CONTINUOUS),
    '58':m.add_var(var_type = CONTINUOUS)
}


In [4]:
# Create the constraints for linking y values

m += (y[0] == d['o0'])
m += (y[1] == d['o0'] + wn[0] + d['01'])
m += (y[2] == d['o0'] + wn[0] + d['01'] + wn[1] + d['12'])
m += (y[3] == d['o1'])
m += (y[4] == d['o1'] + wn[3] + d['34'])
m += (y[5] == d['o1'] + wn[3] + d['34'] + wn[4] + d['45'])
m += (y[6] == d['o2'])
m += (y[7] == d['o2'] + wn[6] + d['67'])
m += (y[8] == d['o2'] + wn[6] + d['67'] + wn[7] + d['78'])

In [5]:
# Straightness constraints

m += (s['03'] >= y[0] - y[3])
m += (s['03'] >= -(y[0] - y[3]))
m += (s['04'] >= y[0] - y[4])
m += (s['04'] >= -(y[0] - y[4]))
m += (s['05'] >= y[0] - y[5])
m += (s['05'] >= -(y[0] - y[5]))
m += (s['14'] >= y[1] - y[4])
m += (s['14'] >= -(y[1] - y[4]))
m += (s['25'] >= y[2] - y[5])
m += (s['25'] >= -(y[2] - y[5]))
m += (s['36'] >= y[3] - y[6])
m += (s['36'] >= -(y[3] - y[6]))
m += (s['37'] >= y[3] - y[7])
m += (s['37'] >= -(y[3] - y[7]))
m += (s['48'] >= y[4] - y[8])
m += (s['48'] >= -(y[4] - y[8]))
m += (s['57'] >= y[5] - y[7])
m += (s['57'] >= -(y[5] - y[7]))
m += (s['58'] >= y[5] - y[8])
m += (s['58'] >= -(y[5] - y[8]))


In [6]:
# Layer constraints

m += (y[1] >= y[0])
m += (y[1] >= y[3])
m += (y[1] >= y[4])
m += (y[1] >= y[6])
m += (y[1] >= y[7])
m += (y[2] >= y[0])
m += (y[2] >= y[3])
m += (y[2] >= y[4])
m += (y[2] >= y[6])
m += (y[2] >= y[7])
m += (y[5] >= y[0])
m += (y[5] >= y[3])
m += (y[5] >= y[4])
m += (y[5] >= y[6])
m += (y[5] >= y[7])
m += (y[8] >= y[0])
m += (y[8] >= y[3])
m += (y[8] >= y[4])
m += (y[8] >= y[6])
m += (y[8] >= y[7])


In [7]:
# Objective function
m.objective = minimize( xsum(s[flow]*w[flow] for flow in s.keys()) )

In [8]:
status = m.optimize()

In [9]:
y[8].x

14.0

In [10]:
y[3].x

0.0

In [11]:
lists = [1,2,3,4,5,6]

for i in range(len(lists)):
    if i+1 != len(lists):
        print(lists[i],lists[i+1])

1 2
2 3
3 4
4 5
5 6


In [12]:
len(lists)

6

In [13]:
for i,node in enumerate(lists):
    print(f'index: {i}, node: {node}')

index: 0, node: 1
index: 1, node: 2
index: 2, node: 3
index: 3, node: 4
index: 4, node: 5
index: 5, node: 6


In [14]:
for i in range(5):
    print(i)

0
1
2
3
4


In [15]:
import ilp_code.dataset_manipulation as dm
import ilp_code.diagram_optimisation as do
import pandas as pd
from floweaver import *
from attr import evolve

In [16]:
# Import the two relevant csv files as dictionaries 
flowst1 = pd.read_csv('datasets\\m3_t1_flows.csv').to_dict('records')
node_deft1 = pd.read_csv('datasets\\m3_t1_nodes.csv').to_dict('records')
datasett1 = Dataset.from_csv('datasets\\m3_t1_flows.csv','datasets\\m3_t1_nodes.csv')

In [17]:
# Generate the dataset required for plotting the diagram
nodest1 = dm.generate_nodes(node_deft1)
orderingt1 = dm.generate_ordering(node_deft1)
orderingt1,nodest1,bundlest1 = dm.generate_waypoints_bundles(node_deft1, flowst1, orderingt1, nodest1)

# Generate sankey data
size = dict(width = 600, height = 500)
sddt1 = SankeyDefinition(nodest1, bundlest1, orderingt1)
sankey_datat1 = weave(sddt1, datasett1)

# Plot diagram
sankey_datat1.to_widget(**size)

SankeyWidget(layout=Layout(height='500', width='600'), links=[{'source': 'n0^*', 'target': 'n3^*', 'type': '*'…

In [18]:
# Optimise the order of the nodes based off crossing area ONLY
modelt1 = do.model_inputs(sankey_datat1)
ordering_optimisedt1 = do.optimise_node_order(modelt1)

# Evolve the diagram and plot optimised version
sankey_data_evolvedt1 = evolve(sankey_datat1, ordering = ordering_optimisedt1)
sankey_data_evolvedt1.to_widget(**size)

SankeyWidget(layout=Layout(height='500', width='600'), links=[{'source': 'n0^*', 'target': 'n3^*', 'type': '*'…

In [19]:
smodel = do.straightness_model(sankey_datat1)

In [20]:
ys = do.optimise_position(smodel)

In [21]:
ys

{'n0^*': 0.0,
 'n1^*': 14.0,
 'n2^*': 23.0,
 'n3^*': 0.0,
 'n4^*': 12.0,
 'n5^*': 23.0,
 'n6^*': 0.0,
 'n7^*': 7.0,
 'n8^*': 14.0}

In [23]:
sankey_data_evolvedt1.to_widget(**size)

SankeyWidget(layout=Layout(height='500', width='600'), links=[{'source': 'n0^*', 'target': 'n3^*', 'type': '*'…

In [78]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout, Output

def show_widget(
        sdd,
        width=700,
        height=500,
        margins=None,
        align_link_types=False,
        link_label_format="",
        link_label_min_width=5,
        debugging=False,
        forceY=None
    ):

        if SankeyWidget is None:
            raise RuntimeError("ipysankeywidget is required")

        if margins is None:
            margins = {
                "top": 25,
                "bottom": 10,
                "left": 130,
                "right": 130,
            }

        value = sdd.to_json(format="widget")
        
        new_nodes = []
        if forceY is not None:
            for node in value['nodes']:
                node['forceY'] = forceY[node['id']]

        widget = SankeyWidget(
            nodes=value["nodes"],
            links=value["links"],
            order=value["order"],
            groups=value["groups"],
            align_link_types=align_link_types,
            linkLabelFormat=link_label_format,
            linkLabelMinWidth=link_label_min_width,
            layout= Layout(width=str(width), height=str(height)),
            margins=margins,
        )
        
        return widget, value['nodes']

In [82]:
w, ns = show_widget(sankey_data_evolvedt1, forceY = ys, height = 500)


In [83]:
w

SankeyWidget(layout=Layout(height='500', width='700'), links=[{'source': 'n0^*', 'target': 'n3^*', 'type': '*'…

In [81]:
ns

[{'id': 'n0^*',
  'title': 'n0',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 0.0},
 {'id': 'n1^*',
  'title': 'n1',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 14.0},
 {'id': 'n2^*',
  'title': 'n2',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 23.0},
 {'id': 'n3^*',
  'title': 'n3',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 0.0},
 {'id': 'n4^*',
  'title': 'n4',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 12.0},
 {'id': 'n5^*',
  'title': 'n5',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 23.0},
 {'id': 'n6^*',
  'title': 'n6',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 0.0},
 {'id': 'n7^*',
  'title': 'n7',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 7.0},
 {'id': 'n8^*',
  'title': 'n8',
  'direction': 'r',
  'hidden': False,
  'type': 'process',
  'forceY': 14.